In [ ]:
import warnings
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge  # noqa
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
warnings.simplefilter(action="ignore", category=FutureWarning)

Task 2.5.1: Write a wrangle function that takes the name of a CSV file as input and returns a DataFrame. The function should do the following steps:

Subset the data in the CSV file and return only apartments in Mexico City ("Distrito Federal") that cost less than $100,000.
Remove outliers by trimming the bottom and top 10% of properties in terms of "surface_covered_in_m2".
Create separate "lat" and "lon" columns.
Mexico City is divided into 15 boroughs. Create a "borough" feature from the "place_with_parent_names" column.
Drop columns that are more than 50% null values.
Drop columns containing low- or high-cardinality categorical values.
Drop any columns that would constitute leakage for the target "price_aprox_usd".
Drop any columns that would create issues of multicollinearity.

In [ ]:
# Build your `wrangle` function
def wrangle(filepath):
    #Read CSV file into DataFrame
    dfs = pd.read_csv(filepath)
    mask_ba = dfs["place_with_parent_names"].str.contains("Distrito Federal")
    mask_apt = dfs["property_type"] == "apartment"
    mask_price = dfs["price_aprox_usd"] < 100000
    dfs = dfs[mask_ba & mask_apt & mask_price]

    low, high = dfs["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = dfs["surface_covered_in_m2"].between(low, high)
    dfs = dfs[mask_area]

    dfs[["lat", "lon"]] = dfs["lat-lon"].str.split(",", expand=True).astype(float)
    dfs.drop(columns="lat-lon", inplace=True)

    dfs["borough"] = dfs["place_with_parent_names"].str.split("|", expand=True)[1]
    dfs.drop(columns="place_with_parent_names", inplace=True)

    dfs.drop(columns=["floor", "expenses", "rooms", "price_usd_per_m2", "surface_total_in_m2"], inplace=True)

    dfs.drop(columns=["operation", "property_type", "currency", "properati_url"], inplace=True)

    dfs.drop(columns=[
        'price',
        'price_aprox_local_currency',
        'price_per_m2'
    ], inplace=True)

    return dfs


Task 2.5.2: Use glob to create the list files. It should contain the filenames of all the Mexico City real estate CSVs in the ./data directory, except for mexico-city-test-features.csv.

In [ ]:
files = glob("data/mexico-city-real-estate-*.csv")
files

Task 2.5.3: Combine your wrangle function, a list comprehension, and pd.concat to create a DataFrame df. It should contain all the properties from the five CSVs in files.

In [ ]:
df = pd.concat([wrangle(file) for file in files], ignore_index=True)
print(df.info())
df.head()

Task 2.5.4: Create a histogram showing the distribution of apartment prices ("price_aprox_usd") in df. Be sure to label the x-axis "Price [$]", the y-axis "Count", and give it the title "Distribution of Apartment Prices". Use Matplotlib (plt).

In [ ]:
# Build histogram

plt.hist(df["price_aprox_usd"])


# Label axes
plt.xlabel("Price [$]")
plt.ylabel("Count")

# Add title
plt.title("Distribution of Apartment Prices")

# Don't delete the code below 👇
plt.savefig("images/2-5-4.png", dpi=150)

Task 2.5.5: Create a scatter plot that shows apartment price ("price_aprox_usd") as a function of apartment size ("surface_covered_in_m2"). Be sure to label your x-axis "Area [sq meters]" and y-axis "Price [USD]". Your plot should have the title "Mexico City: Price vs. Area". Use Matplotlib (plt).

In [ ]:
# Build scatter plot
plt.scatter(x=df["surface_covered_in_m2"], y=df["price_aprox_usd"])


# Label axes
plt.xlabel("Area [sq meters]")
plt.ylabel("Price [USD]")

# Add title
plt.title("Mexico City: Price vs. Area")


# Don't delete the code below 👇
plt.savefig("images/2-5-5.png", dpi=150)


Task 2.5.6: Create a Mapbox scatter plot that shows the location of the apartments in your dataset and represent their price using color.

In [ ]:
# Plot Mapbox location and price
fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    width=600,
    height=600,
    color="price_aprox_usd",
    hover_data=["price_aprox_usd"]
)

fig.update_layout(mapbox_style="open-street-map")

fig.show()

Task 2.5.7: Create your feature matrix X_train and target vector y_train. Your target is "price_aprox_usd". Your features should be all the columns that remain in the DataFrame you cleaned above.

In [ ]:
# Split data into feature matrix `X_train` and target vector `y_train`.
target = "price_aprox_usd"
features = ["surface_covered_in_m2", "lat", "lon", "borough"]
X_train = df[features]
y_train = df[target]


Task 2.5.8: Calculate the baseline mean absolute error for your model.

In [ ]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] * len(y_train)
baseline_mae = mean_absolute_error(y_train, y_pred_baseline)
print("Mean apt price:", y_mean)
print("Baseline MAE:", baseline_mae)

Task 2.5.9: Create a pipeline named model that contains all the transformers necessary for this dataset and one of the predictors you've used during this project. Then fit your model to the training data.

In [ ]:
# Build Model
model = make_pipeline(OneHotEncoder(use_cat_names=True), SimpleImputer(), Ridge())

# Fit model
model.fit(X_train, y_train)

Task 2.5.10: Read the CSV file mexico-city-test-features.csv into the DataFrame X_test.

In [ ]:
X_test = pd.read_csv("data/mexico-city-test-features.csv")
print(X_test.info())
X_test.head()

Task 2.5.11: Use your model to generate a Series of predictions for X_test. When you submit your predictions to the grader, it will calculate the mean absolute error for your model.

In [ ]:
y_test_pred = pd.Series(model.predict(X_test))
y_test_pred.head()

Task 2.5.12: Create a Series named feat_imp. The index should contain the names of all the features your model considers when making predictions; the values should be the coefficient values associated with each feature. The Series should be sorted ascending by absolute value.

In [ ]:
coefficients = model.named_steps["ridge"].coef_
features = model.named_steps["onehotencoder"].get_feature_names()
feat_imp = pd.Series(coefficients, index=features).sort_values(key=abs)
feat_imp

Task 2.5.13: Create a horizontal bar chart that shows the 10 most influential coefficients for your model. Be sure to label your x- and y-axis "Importance [USD]" and "Feature", respectively, and give your chart the title "Feature Importances for Apartment Price". Use pandas.

In [ ]:
# Build bar chart
feat_imp.tail(10).plot(kind="barh")


# Label axes
plt.xlabel("Importance [USD]")
plt.ylabel("Feature")

# Add title
plt.title("Feature Importance for Apartment Price")


# Don't delete the code below 👇
plt.savefig("images/2-5-13.png", dpi=150)